# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [87]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [88]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [90]:
stock = yf.Ticker('AAPL')
data = stock.info
data

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod

In [91]:
print(f"Current or Latest Price of the stock: {data['currentPrice']}")
print(f"1 year Change: {data['52WeekChange']}")

Current or Latest Price of the stock: 221.27
1 year Change: 0.22586644


In [92]:
# Initialize an empty list to store the results
data = []

# Iterate through each stock symbol in the CSV file
for symbol in stocks['Ticker']:
    try:
        # Fetch the stock data using yfinance
        stock = yf.Ticker(symbol)
        stock_info = stock.info

        # Get the stock price and market cap
        stock_price = stock_info.get('currentPrice', None)
        one_year_price_return = stock_info.get('52WeekChange', None)

        # Append the data to the list
        data.append([symbol, stock_price, one_year_price_return, 0])

    except Exception as e:
        print(f"Failed to get data for {symbol}: {e}")
        data.append([symbol, None, None, 0])

# Create a DataFrame from the collected data
data = pd.DataFrame(data, columns= ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy'])

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=RNEJFNig9cq


Failed to get data for AZO: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FLT&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PEAK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PEAK&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PXD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.c

Failed to get data for ZBRA: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out.


In [139]:
data.isna().sum()

Ticker                      0
Price                      44
One-Year Price Return      46
Number of Shares to Buy     0
dtype: int64

In [140]:
final_dataframe = data.copy(deep=True)

In [141]:
final_dataframe.isna().sum()

Ticker                      0
Price                      44
One-Year Price Return      46
Number of Shares to Buy     0
dtype: int64

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [142]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,116.14,1.481111,0
1,NRG,81.86,1.161570,0
2,GPS,23.11,1.105513,0
3,HWM,93.92,0.903424,0
4,SLG,62.23,0.885200,0
5,ANET,346.19,0.883273,0
6,GE,167.96,0.836523,0
7,IRM,107.89,0.821183,0
8,PGR,222.33,0.783207,0
9,AVGO,156.16,0.763994,0


In [143]:
final_dataframe.isna().sum()

Ticker                     0
Price                      0
One-Year Price Return      0
Number of Shares to Buy    0
dtype: int64

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [144]:
portfolio_size = 100000

In [145]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,116.14,1.481111,16
1,NRG,81.86,1.161570,23
2,GPS,23.11,1.105513,84
3,HWM,93.92,0.903424,20
4,SLG,62.23,0.885200,31
5,ANET,346.19,0.883273,5
6,GE,167.96,0.836523,11
7,IRM,107.89,0.821183,18
8,PGR,222.33,0.783207,8
9,AVGO,156.16,0.763994,12


In [146]:
ticker = 'AAPL'
stock_data = yf.Ticker(ticker)

# Get historical market data (1 year)
hist = stock_data.history(period="1y")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-14 00:00:00-04:00,177.063967,178.775212,176.407323,178.546387,43675600,0.00,0.0
2023-08-15 00:00:00-04:00,177.969313,178.566249,176.148628,176.546585,43622600,0.00,0.0
2023-08-16 00:00:00-04:00,176.228228,177.631038,175.601431,175.671082,46964900,0.00,0.0
2023-08-17 00:00:00-04:00,176.238180,176.606291,172.596809,173.114166,66062900,0.00,0.0
2023-08-18 00:00:00-04:00,171.422831,174.208580,171.084566,173.601685,61114200,0.00,0.0
...,...,...,...,...,...,...,...
2024-08-07 00:00:00-04:00,206.660786,213.392999,206.151381,209.577423,63516400,0.00,0.0
2024-08-08 00:00:00-04:00,212.863619,213.952355,208.588569,213.063385,47161100,0.00,0.0
2024-08-09 00:00:00-04:00,211.854792,216.529374,211.724938,215.990005,42201600,0.00,0.0


In [147]:
print(hist['Open'].iloc[0])
print(hist['Close'].iloc[0])

print(hist['Open'].iloc[-1])
print(hist['Close'].iloc[-1])

177.06396675305982
178.54638671875
219.10499572753906
221.27000427246094


In [148]:
(hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]

0.23928581439740965

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [149]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

In [150]:
# Assuming final_df is your DataFrame
final_dataframe = final_dataframe.dropna()

# Reset the index and drop the old index column
final_dataframe = final_dataframe.reset_index(drop=True)

In [151]:
# Store returns for percentile calculation
all_returns = {
    'One-Year': [],
    'Six-Month': [],
    'Three-Month': [],
    'One-Month': []
}

# Fetch data and calculate metrics for each ticker
for i in range(len(final_dataframe)):
    ticker = final_dataframe['Ticker'][i]
    stock_data = yf.Ticker(ticker)
    
    # Get historical market data
    hist = stock_data.history(period="1y")
    
    # Initialize returns
    one_year_return = None
    six_month_return = None
    three_month_return = None
    one_month_return = None
    
    # Check if enough data is available
    if len(hist) < 30:  # e.g., need at least 30 days of data for calculations
        continue

    # Calculate returns
    # if len(hist) >= 252:  # Approx. 252 trading days in a year
    #     one_year_return = (hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]
    #     final_dataframe.loc[i, 'One-Year Price Return'] = one_year_return
    #     all_returns['One-Year'].append(one_year_return)

    if len(hist) >= 126:  # Approx. 126 trading days in six months
        six_month_return = (hist['Close'][-1] - hist['Close'][-126]) / hist['Close'][-126]
        final_dataframe.loc[i, 'Six-Month Price Return'] = six_month_return
        all_returns['Six-Month'].append(six_month_return)

    if len(hist) >= 63:  # Approx. 63 trading days in three months
        three_month_return = (hist['Close'][-1] - hist['Close'][-63]) / hist['Close'][-63]
        final_dataframe.loc[i, 'Three-Month Price Return'] = three_month_return
        all_returns['Three-Month'].append(three_month_return)

    if len(hist) >= 21:  # Approx. 21 trading days in one month
        one_month_return = (hist['Close'][-1] - hist['Close'][-21]) / hist['Close'][-21]
        final_dataframe.loc[i, 'One-Month Price Return'] = one_month_return
        all_returns['One-Month'].append(one_month_return)

    # Example HQM Score calculation (replace with actual logic)
    # For now, set to 0 as placeholder
    final_dataframe.loc[i, 'HQM Score'] = 0.0

# Calculate percentiles
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

# for time_period in time_periods:
#     percentile_column = f'{time_period} Return Percentile'
#     return_column = f'{time_period} Price Return'
#     final_dataframe[percentile_column] = final_dataframe.apply(
#         lambda row: stats.percentileofscore(all_returns[time_period], row[return_column]) / 100,
#         axis=1
    # )

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(final_dataframe[f'{time_period} Return Percentile'])

/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/221350625.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  six_month_return = (hist['Close'][-1] - hist['Close'][-126]) / hist['Close'][-126]
/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/221350625.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  three_month_return = (hist['Close'][-1] - hist['Close'][-63]) / hist['Close'][-63]
/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/221350625.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alway

In [152]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,HQM Score
0,NVDA,116.14,1.481111,16,0.610401,0.271395,-0.080880,0.0
1,NRG,81.86,1.161570,23,0.594056,0.002762,0.076890,0.0
2,GPS,23.11,1.105513,84,0.184091,0.080852,-0.039485,0.0
3,HWM,93.92,0.903424,20,0.517579,0.162238,0.155097,0.0
4,SLG,62.23,0.885200,31,0.481209,0.151177,-0.012278,0.0
5,ANET,346.19,0.883273,5,0.303868,0.103711,-0.033610,0.0
6,GE,167.96,0.836523,11,0.489632,0.051527,0.031379,0.0
7,IRM,107.89,0.821183,18,0.635063,0.349716,0.079116,0.0
8,PGR,222.33,0.783207,8,0.222217,0.041023,0.058714,0.0
9,AVGO,156.16,0.763994,12,0.256925,0.135163,-0.078049,0.0


In [153]:
print(final_dataframe.isna().sum())

Ticker                      0
Price                       0
One-Year Price Return       0
Number of Shares to Buy     0
Six-Month Price Return      1
Three-Month Price Return    1
One-Month Price Return      1
HQM Score                   1
dtype: int64


In [154]:
# Assuming final_dataframe is your DataFrame
final_dataframe = final_dataframe.dropna()

# Reset the index and drop the old index column
final_dataframe = final_dataframe.reset_index(drop=True)

In [157]:
# Find rows where any value is NaN
rows_with_nan = final_dataframe[final_dataframe.isna().any(axis=1)]

# Print rows with NaN values
print(rows_with_nan)

Empty DataFrame
Columns: [Ticker, Price, One-Year Price Return, Number of Shares to Buy, Six-Month Price Return, Three-Month Price Return, One-Month Price Return, HQM Score]
Index: []


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [158]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in final_dataframe.index:
    for time_period in time_periods:
        final_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(final_dataframe[f'{time_period} Price Return'], final_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(final_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
final_dataframe

0     1.00
1     0.98
2     0.96
3     0.94
4     0.92
5     0.90
6     0.88
7     0.86
8     0.84
9     0.82
10    0.80
11    0.78
12    0.76
13    0.74
14    0.72
15    0.70
16    0.68
17    0.66
18    0.64
19    0.62
20    0.60
21    0.58
22    0.56
23    0.54
24    0.52
25    0.50
26    0.48
27    0.46
28    0.44
29    0.42
30    0.40
31    0.38
32    0.36
33    0.34
34    0.32
35    0.30
36    0.28
37    0.26
38    0.24
39    0.22
40    0.20
41    0.18
42    0.16
43    0.14
44    0.12
45    0.10
46    0.08
47    0.06
48    0.04
49    0.02
Name: One-Year Return Percentile, dtype: float64
0     0.96
1     0.94
2     0.28
3     0.92
4     0.88
5     0.72
6     0.90
7     0.98
8     0.48
9     0.60
10    0.80
11    0.66
12    0.84
13    0.08
14    0.52
15    0.86
16    0.54
17    0.44
18    0.58
19    0.62
20    0.42
21    0.78
22    0.56
23    0.02
24    0.50
25    0.30
26    0.24
27    0.76
28    0.38
29    0.40
30    0.36
31    0.14
32    0.22
33    0.70
34    0.26
35    0.20
36   

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,HQM Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,NVDA,116.14,1.481111,16,0.610401,0.271395,-0.080880,0.0,1.00,0.96,0.98,0.16
1,NRG,81.86,1.161570,23,0.594056,0.002762,0.076890,0.0,0.98,0.94,0.18,0.86
2,GPS,23.11,1.105513,84,0.184091,0.080852,-0.039485,0.0,0.96,0.28,0.56,0.32
3,HWM,93.92,0.903424,20,0.517579,0.162238,0.155097,0.0,0.94,0.92,0.82,0.96
4,SLG,62.23,0.885200,31,0.481209,0.151177,-0.012278,0.0,0.92,0.88,0.78,0.60
5,ANET,346.19,0.883273,5,0.303868,0.103711,-0.033610,0.0,0.90,0.72,0.66,0.38
6,GE,167.96,0.836523,11,0.489632,0.051527,0.031379,0.0,0.88,0.90,0.40,0.72
7,IRM,107.89,0.821183,18,0.635063,0.349716,0.079116,0.0,0.86,0.98,1.00,0.88
8,PGR,222.33,0.783207,8,0.222217,0.041023,0.058714,0.0,0.84,0.48,0.38,0.80
9,AVGO,156.16,0.763994,12,0.256925,0.135163,-0.078049,0.0,0.82,0.60,0.74,0.20


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [159]:
from statistics import mean

for row in final_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(final_dataframe.loc[row, f'{time_period} Return Percentile'])
    final_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [160]:
final_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = final_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [161]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,HQM Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,NVDA,116.14,1.481111,17,0.610401,0.271395,-0.080880,0.775,1.00,0.96,0.98,0.16
1,NRG,81.86,1.161570,24,0.594056,0.002762,0.076890,0.740,0.98,0.94,0.18,0.86
2,GPS,23.11,1.105513,86,0.184091,0.080852,-0.039485,0.530,0.96,0.28,0.56,0.32
3,HWM,93.92,0.903424,21,0.517579,0.162238,0.155097,0.910,0.94,0.92,0.82,0.96
4,SLG,62.23,0.885200,32,0.481209,0.151177,-0.012278,0.795,0.92,0.88,0.78,0.60
5,ANET,346.19,0.883273,5,0.303868,0.103711,-0.033610,0.665,0.90,0.72,0.66,0.38
6,GE,167.96,0.836523,11,0.489632,0.051527,0.031379,0.725,0.88,0.90,0.40,0.72
7,IRM,107.89,0.821183,18,0.635063,0.349716,0.079116,0.930,0.86,0.98,1.00,0.88
8,PGR,222.33,0.783207,8,0.222217,0.041023,0.058714,0.625,0.84,0.48,0.38,0.80
9,AVGO,156.16,0.763994,12,0.256925,0.135163,-0.078049,0.590,0.82,0.60,0.74,0.20


In [169]:
hqm_dataframe.to_csv('hqm_dataframe.csv')